### I claim no credit for the CSV's. many thanks to the respective authers who wrote them. 
(https://www.kaggle.com/redwankarimsony/melanoma-submissions2?select=submission_multiple_data_source.csv) 
(https://www.kaggle.com/ragnar123/rank-then-blend) 
(https://www.kaggle.com/shivam17818/test-all-efficientnet-model-b0-b7/output)
(https://www.kaggle.com/truonghoang/stacking-ensemble-on-my-submissions/output)

Ensemble learning is an excellent machine learning idea which displays noticeable benefits in many applications, one such notable example is the widespread use of ensembles in Kaggle competitions. In an ensemble several individual models (for instance ResNet18 and VGG16) which were trained on the same corpus, work in tandem and during inference, their predictions are fused by a pre-defined strategy to yield a single prediction. 


The code below was adapted from something i wrote few years ago (https://github.com/QuantScientist/Deep-Learning-Boot-Camp/blob/master/Kaggle-PyTorch/PyTorch-Ensembler/utils.py) 

You can set up cutoff thresholds for the lower and upper bounds with this:

    cutoff_lo = 0.88
    cutoff_hi = 0.11


In [ ]:
import numpy as np
import pandas as pd 
import os 

def MinMaxBestBaseStacking(input_folder, best_base, output_path):
    sub_base = pd.read_csv(best_base)
    all_files = os.listdir(input_folder)

    # Read and concatenate submissions
    outs = [pd.read_csv(os.path.join(input_folder, f), index_col=0) for f in all_files]
    concat_sub = pd.concat(outs, axis=1)
    cols = list(map(lambda x: "target" + str(x), range(len(concat_sub.columns))))
    concat_sub.columns = cols
    concat_sub.reset_index(inplace=True)

    # get the data fields ready for stacking
    concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
    concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
    concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
    concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

    # set up cutoff threshold for lower and upper bounds
    cutoff_lo = 0.73
    cutoff_hi = 0.33

    concat_sub['is_iceberg_base'] = sub_base['target']
    concat_sub['target'] = np.where(np.all(concat_sub.iloc[:, 1:6] > cutoff_lo, axis=1),
                                        concat_sub['is_iceberg_max'],
                                        np.where(np.all(concat_sub.iloc[:, 1:6] < cutoff_hi, axis=1),
                                                 concat_sub['is_iceberg_min'],
                                                 concat_sub['is_iceberg_base']))
    concat_sub[['image_name', 'target']].to_csv(output_path,
                                            index=False, float_format='%.12f')
    
# def ensemble():
#     stacked_0 = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
    
#     stacked_1 = pd.read_csv('/kaggle/input/melanoma-submissions2/submissionImage.csv')
#     stacked_2 = pd.read_csv('/kaggle/input/melanoma-submissions2/submissionTabular.csv')
#     stacked_4 = pd.read_csv('/kaggle/input/melanoma-submissions2/submission_multiple_data_source.csv')
#     stacked_3 = pd.read_csv('../input/csv001/lgbm_baseline_sub.csv')    
#     stacked_5 = pd.read_csv('../input/csv001/sub77.csv')
#     stacked_6 = pd.read_csv('../input/csv001/submission.csv')
#     stacked_7 = pd.read_csv('../input/csv001/submission_models_blended.csv')
    
#     sub = pd.DataFrame()
#     sub['image_name'] = stacked_0['image_name']
#     sub['target'] = np.exp(np.mean(
#         [
#             stacked_1['target'].apply(lambda x: np.log2(x)), \
#             stacked_2['target'].apply(lambda x: np.log2(x)), \
#             stacked_4['target'].apply(lambda x: np.log2(x)), \
#             stacked_3['target'].apply(lambda x: np.log2(x)), 
#             stacked_5['target'].apply(lambda x: np.log2(x)), \
#             stacked_6['target'].apply(lambda x: np.log2(x)), \
#             stacked_7['target'].apply(lambda x: np.log2(x)),           
#             ], axis=0))
#     sub.to_csv('submission.csv', index=False, float_format='%.6f')

In [ ]:
# ensemble()
# sub.to_csv('submission.csv', index = False)

MinMaxBestBaseStacking('../input/csv001/', '../input/base001/datasets_766461_1344911_submission_rank-then-blend.csv', 'submission.csv') # 0.9526 
# MinMaxBestBaseStacking('../input/csv001/', '../input/csv001/submission_efnet.csv', 'submission.csv')
# MinMaxBestBaseStacking('../input/csv001/', '../input/csv001/submission0.9504.csv', 'submission.csv')




